In [1]:
# Importing the TfidfVectorizer from sklearn to convert text to TF-IDF vectors
from sklearn.feature_extraction.text import TfidfVectorizer
# Importing the NearestNeighbors from sklearn to create the nearest neighbor index
from sklearn.neighbors import NearestNeighbors
import pandas as pd

# List of documents to be processed
documents = [
    "This is the `Fundamentals of RAG course`",
    "Educative is an AI-powered online learning platform",
    "There are several Generative AI courses available on Educative",
    "I am writing this using my keyboard"
]

# Initializing TfidfVectorizer to remove English stop words
vectorizer = TfidfVectorizer(stop_words='english')

# Transforming the documents into a matrix of TF-IDF features
tfidf_matrix = vectorizer.fit_transform(documents)

# Create a DataFrame to display the TF-IDF matrix more readably
feature_names = vectorizer.get_feature_names_out()
df = pd.DataFrame(tfidf_matrix.toarray(), columns=feature_names, index=[f"Doc {i+1}" for i in range(len(documents))])

# Print the TF-IDF matrix using DataFrame for better formatting
print("--" * 50)
print("This is how it looks after being vectorized:\n")
print("TF-IDF Matrix:")
print(df)

# Importing NearestNeighbors from sklearn for creating the nearest neighbor index
# This module is used to efficiently find the closest vector(s) in high-dimensional space, which is crucial for the retrieval functionality in our RAG system

# Initializing NearestNeighbors to create a conceptual vector database (index) for the RAG system
# This index, utilizing cosine similarity, functions effectively as the vector database, storing all document vectors and enabling their retrieval based on similarity measures
index = NearestNeighbors(n_neighbors=1, metric='cosine').fit(tfidf_matrix)
print(index)

# Function to query the index with a new document/query
def query_index(query):
    # Transforming the query into the same TF-IDF vector space as the documents
    query_vec = vectorizer.transform([query])
    print(f"Query Vector for '{query}':")
    print(query_vec.toarray())

    # Finding the nearest neighbor to the query vector in the index
    distance, indices = index.kneighbors(query_vec)
    print("Nearest document index:", indices[0][0])
    print("Distance from query:", distance[0][0])

    return documents[indices[0][0]]


# Example query to test the indexing and retrieval system
query = "What course is this?"
result = query_index(query)

# Printing the document retrieved as the closest match to the query
print("--" * 50)
print("Retrieved document:", result)


----------------------------------------------------------------------------------------------------
This is how it looks after being vectorized:

TF-IDF Matrix:
             ai  available   course   courses  educative  fundamentals  \
Doc 1  0.000000   0.000000  0.57735  0.000000   0.000000       0.57735   
Doc 2  0.344315   0.000000  0.00000  0.000000   0.344315       0.00000   
Doc 3  0.382743   0.485461  0.00000  0.485461   0.382743       0.00000   
Doc 4  0.000000   0.000000  0.00000  0.000000   0.000000       0.00000   

       generative  keyboard  learning    online  platform   powered      rag  \
Doc 1    0.000000   0.00000  0.000000  0.000000  0.000000  0.000000  0.57735   
Doc 2    0.000000   0.00000  0.436719  0.436719  0.436719  0.436719  0.00000   
Doc 3    0.485461   0.00000  0.000000  0.000000  0.000000  0.000000  0.00000   
Doc 4    0.000000   0.57735  0.000000  0.000000  0.000000  0.000000  0.00000   

         using  writing  
Doc 1  0.00000  0.00000  
Doc 2  0.00000

In [ ]:
import numpy as np
import openai
openai.api_key = '{{OpenAI_API}}'  # Set your OpenAI API key here

from sklearn.neighbors import NearestNeighbors


def get_gpt4_embedding(text):
    response = openai.embeddings.create(
        input=text,
        model="text-embedding-3-small"
    )
    # Access the embedding directly from the response object
    return response.data[0].embedding
  
# List of example documents to be used in the index
documents = [
    "This is the Fundamentals of RAG course.",
    "Educative is an AI-powered online learning platform.",
    "There are several Generative AI courses available on Educative.",
    "I am writing this using my keyboard.",
    "JavaScript is a good programming language :)"
]

# Get embeddings for each document using the get_gpt4_embedding function
embeddings = [get_gpt4_embedding(doc) for doc in documents]
embeddings = np.array(embeddings)
print("--" * 50)
print("This is how it looks after going through an embedding model:\n")
print(embeddings)

# Fit a NearestNeighbors model on the document embeddings using cosine similarity
index = NearestNeighbors(n_neighbors=1, metric='cosine').fit(embeddings)

# Function to query the index with a given text query


def query_index(query):
    query_embedding = get_gpt4_embedding(query)
    query_embedding = np.array([query_embedding])
    distance, indices = index.kneighbors(query_embedding)
    return documents[indices[0][0]]

# Example Query
query = "What is JS?"
print("Query:", query)
result = query_index(query)  # Retrieve the most similar document to the query

print("Retrieved document:", result)  # Print the retrieved document